In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from cs_6804_project.src.torch_cloudnet.model import CloudNet
from cs_6804_project.src.keras_cloudnet.utils import get_input_image_names

In [2]:
# Paths to data
GLOBAL_PATH = "D:/38-Cloud/"
TRAIN_FOLDER = os.path.join(GLOBAL_PATH, '38-Cloud_training')
TEST_FOLDER = os.path.join(GLOBAL_PATH, '38-Cloud_test')

# Set params
in_rows = 192
in_cols = 192
num_of_channels = 4
num_of_classes = 1
starting_learning_rate = 1e-4
end_learning_rate = 1e-8
max_num_epochs = 2000  # just a huge number. The actual training should not be limited by this value
val_ratio = 0.2
patience = 15
decay_factor = 0.7
batch_sz = 12
max_bit = 65535  # maximum gray level in landsat 8 images
experiment_name = "Cloud-Net"
weights_path = os.path.join(GLOBAL_PATH, experiment_name + '.h5')
train_resume = False

100%|██████████████████████████████████████████████████████████████████████████| 5155/5155 [00:00<00:00, 396305.53it/s]


In [ ]:
# Get input/target image names
train_patches_csv_name = 'training_patches_38-cloud_nonempty.csv'
df_train_img = pd.read_csv(os.path.join(GLOBAL_PATH, train_patches_csv_name))
train_img, train_msk = get_input_image_names(df_train_img, TRAIN_FOLDER, if_train=True)

In [ ]:
# Split data into training and validation
train_img_split, val_img_split, train_msk_split, val_mask_split = train_test_split(train_img, train_msk,
                                                                                   test_size=val_ratio,
                                                                                   random_state=42, shuffle=True)

In [ ]:
# Perform image augmentation
def generate_augmented_dataset(train, target, in_rows, in_cols, batch_sz, max_bit):
    